In [3]:
# Install all required libraries
!pip install scikit-learn flwr qiskit numpy pandas matplotlib joblib pennylane

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import joblib
import os
import pennylane as qml
import random
import base64
from cryptography.fernet import Fernet
from time import sleep
from tqdm import tqdm

# -------------------- Configuration --------------------
ROLE = "client"  # Change to "server" when running on the server
CLIENT_ID = 1  # Unique ID for the client
DATASET_PATH = "client_1_dataset.csv"
TEST_DATASET_PATH = "client_2_dataset.csv"  # Path to your testing datasetjbvubdiubvdbvsyubvsuyvvbxsuyvuivuifvtuivctuvu
SERVER_MODEL_PATH = "server_global_model.joblib"
LOCAL_MODEL_PATH = f"client_{CLIENT_ID}_model_encrypted.joblib"
ENCRYPTION_KEY_PATH = "encryption_key.key"
DECRYPTED_MODEL_PATH = f"client_{CLIENT_ID}_model_decrypted.joblib"
NOISE_LEVEL = 0.001  # Quantum noise level
NUM_QUBITS = 4

# Quantum Device
dev = qml.device("default.mixed", wires=NUM_QUBITS)

# -------------------- Quantum Differential Privacy --------------------
@qml.qnode(dev)
def quantum_noise(weights):
    """Simulate quantum noise."""
    for i in range(NUM_QUBITS):
        qml.RX(weights[i], wires=i)
        qml.RY(weights[i], wires=i)
    qml.DepolarizingChannel(NOISE_LEVEL, wires=0)
    qml.PhaseDamping(NOISE_LEVEL, wires=1)
    return [qml.expval(qml.PauliZ(i)) for i in range(NUM_QUBITS)]

def apply_qdp(gradients):
    """Apply Quantum Differential Privacy."""
    noisy_gradients = []
    for grad in gradients:
        weights = np.random.rand(NUM_QUBITS)
        noise = sum(quantum_noise(weights)) / NUM_QUBITS
        noisy_gradients.append(grad + noise)
    return np.array(noisy_gradients)

# -------------------- Encryption Utilities --------------------
def load_or_create_key():
    if os.path.exists(ENCRYPTION_KEY_PATH):
        with open(ENCRYPTION_KEY_PATH, "rb") as file:
            return file.read()
    else:
        key = Fernet.generate_key()
        with open(ENCRYPTION_KEY_PATH, "wb") as file:
            file.write(key)
        return key

def encrypt_model(model_path, encrypted_path, key):
    """Encrypt the model file."""
    fernet = Fernet(key)
    with open(model_path, "rb") as file:
        encrypted_data = fernet.encrypt(file.read())
    with open(encrypted_path, "wb") as file:
        file.write(encrypted_data)

def decrypt_model(encrypted_path, decrypted_path, key):
    """Decrypt the model file."""
    fernet = Fernet(key)
    with open(encrypted_path, "rb") as file:
        decrypted_data = fernet.decrypt(file.read())
    with open(decrypted_path, "wb") as file:
        file.write(decrypted_data)

# -------------------- Load and Preprocess Data --------------------
def load_and_preprocess_data(train=True):
    """
    Load and preprocess data.
    If train=True, load the training dataset. Else, load the testing dataset.
    """
    processed_file = "preprocessed_data_train.joblib" if train else "preprocessed_data_test.joblib"
    dataset_path = DATASET_PATH if train else TEST_DATASET_PATH

    if os.path.exists(processed_file):
        print(f"Loading preprocessed {'training' if train else 'testing'} data...")
        data = joblib.load(processed_file)
        return data["X"], data["y"]

    # Load the dataset
    print(f"Loading {'training' if train else 'testing'} dataset...")
    df = pd.read_csv(dataset_path)

    # Drop forbidden and irrelevant columns
    df = df.drop(columns=['oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest'], errors='ignore')
    df = df.drop(columns=['nameOrig', 'nameDest'], errors='ignore')

    # One-Hot Encoding
    df = pd.get_dummies(df, columns=['type'], drop_first=True)

    # Handle missing values
    imputer = SimpleImputer(strategy="mean")
    df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

    # Binary target column
    df_imputed['isFraud'] = df_imputed['isFraud'].apply(lambda x: 1 if x > 0 else 0)

    # Features and target
    X = df_imputed.drop(columns=['isFraud'])
    y = df_imputed['isFraud']

    # Scale numerical features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Save preprocessed data
    joblib.dump({"X": X_scaled, "y": y}, processed_file)
    print(f"Preprocessed {'training' if train else 'testing'} data saved!")

    return X_scaled, y

# -------------------- Client Node --------------------
from sklearn.metrics import precision_score, recall_score, f1_score
from tqdm import tqdm  # For progress bar

def client_node():
    print(f"Client {CLIENT_ID}: Starting...")
    global_model_exists = os.path.exists(SERVER_MODEL_PATH)

    if global_model_exists:
        print(f"Client {CLIENT_ID}: Loading Global Model from Server...")
        global_model = joblib.load(SERVER_MODEL_PATH)
    else:
        print(f"Client {CLIENT_ID}: No Global Model Found. Training Fresh Model...")
        global_model = None

    # Load and preprocess training data
    print("Loading training dataset...")
    X_train, y_train = load_and_preprocess_data()
    joblib.dump({"X": X_train, "y": y_train}, "train_data_preprocessed.joblib")
    print("Preprocessed training data saved!")

    # Load and preprocess testing data
    print("Loading testing dataset...")
    TEST_DATASET_PATH = "test_dataset.csv"  # Path to the testing dataset
    X_test, y_test = load_and_preprocess_data()
    joblib.dump({"X": X_test, "y": y_test}, "test_data_preprocessed.joblib")
    print("Preprocessed testing data saved!")

    # Initialize or load the model
    model = RandomForestClassifier(n_estimators=10, warm_start=True, random_state=42)

    # Training Loop with Epochs
    num_epochs = 5
    for epoch in range(1, num_epochs + 1):
        print(f"Client {CLIENT_ID}: Epoch {epoch} - Training...")

        # Incrementally train the model
        model.set_params(n_estimators=epoch * 10)  # Add 10 trees per epoch
        model.fit(X_train, y_train)

        # Evaluate the model on the training dataset
        y_pred_train = model.predict(X_train)
        accuracy = accuracy_score(y_train, y_pred_train)
        precision = precision_score(y_train, y_pred_train)
        recall = recall_score(y_train, y_pred_train)
        f1 = f1_score(y_train, y_pred_train)

        # Print performance metrics
        print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

    # Final Test on the Testing Dataset
    print("Client {CLIENT_ID}: Testing Local Model...")
    y_pred_test = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred_test)
    print(f"Client {CLIENT_ID}: Final Testing Accuracy: {test_accuracy:.4f}")

    # Save the model
    joblib.dump(model, LOCAL_MODEL_PATH)
    print(f"Client {CLIENT_ID}: Local Model Saved Successfully!")

# -------------------- Server Node --------------------
def server_node():
    key = load_or_create_key()
    print("Server: Starting Aggregation...")
    models = []
    for client_id in range(1, 4):  # Assume 3 clients
        encrypted_model_path = f"client_{client_id}_model_encrypted.joblib"
        if os.path.exists(encrypted_model_path):
            decrypt_model(encrypted_model_path, DECRYPTED_MODEL_PATH, key)
            model = joblib.load(DECRYPTED_MODEL_PATH)
            models.append(model)
            print(f"Server: Model from Client {client_id} Loaded.")
    if not models:
        print("Server: No models found!")
        return

    # Aggregate models
    avg_importances = np.mean([model.feature_importances_ for model in models], axis=0)
    global_model = RandomForestClassifier(n_estimators=100, random_state=42)
    global_model.fit(np.zeros((10, len(avg_importances))), np.zeros(10))  # Dummy data
    global_model.feature_importances_ = avg_importances

    # Save global model
    joblib.dump(global_model, SERVER_MODEL_PATH)
    encrypt_model(SERVER_MODEL_PATH, SERVER_MODEL_PATH, key)
    print("Server: Global Model Aggregated and Saved.")

# -------------------- Execution --------------------
if ROLE == "client":
    client_node()
elif ROLE == "server":
    server_node()

Client 1: Starting...
Client 1: No Global Model Found. Training Fresh Model...
Loading training dataset...
Loading preprocessed training data...
Preprocessed training data saved!
Loading testing dataset...
Loading preprocessed training data...
Preprocessed testing data saved!
Client 1: Epoch 1 - Training...


In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import joblib
import os
import pennylane as qml
import random
from time import sleep
from tqdm import tqdm

# -------------------- Configuration --------------------
ROLE = "client"  # Change to "server" when running on the server
CLIENT_ID = 1  # Unique ID for the client
DATASET_PATH = "client_1_dataset.csv"
TEST_DATASET_PATH = "client_2_dataset.csv"  # Path to your testing dataset
SERVER_MODEL_PATH = "server_global_model.joblib"
LOCAL_MODEL_PATH = f"client_{CLIENT_ID}_model.joblib"  # No encryption for local model
NOISE_LEVEL = 0.001  # Quantum noise level
NUM_QUBITS = 4

# Quantum Device
dev = qml.device("default.mixed", wires=NUM_QUBITS)

# -------------------- Quantum Differential Privacy --------------------
@qml.qnode(dev)
def quantum_noise(weights):
    """Simulate quantum noise."""
    for i in range(NUM_QUBITS):
        qml.RX(weights[i], wires=i)
        qml.RY(weights[i], wires=i)
    qml.DepolarizingChannel(NOISE_LEVEL, wires=0)
    qml.PhaseDamping(NOISE_LEVEL, wires=1)
    return [qml.expval(qml.PauliZ(i)) for i in range(NUM_QUBITS)]

def apply_qdp(gradients):
    """Apply Quantum Differential Privacy."""
    noisy_gradients = []
    for grad in gradients:
        weights = np.random.rand(NUM_QUBITS)
        noise = sum(quantum_noise(weights)) / NUM_QUBITS
        noisy_gradients.append(grad + noise)
    return np.array(noisy_gradients)

# -------------------- Load and Preprocess Data --------------------
def load_and_preprocess_data(train=True):
    """
    Load and preprocess data.
    If train=True, load the training dataset. Else, load the testing dataset.
    """
    processed_file = "preprocessed_data_train.joblib" if train else "preprocessed_data_test.joblib"
    dataset_path = DATASET_PATH if train else TEST_DATASET_PATH

    if os.path.exists(processed_file):
        print(f"Loading preprocessed {'training' if train else 'testing'} data...")
        data = joblib.load(processed_file)
        return data["X"], data["y"]

    # Load the dataset
    print(f"Loading {'training' if train else 'testing'} dataset...")
    df = pd.read_csv(dataset_path)

    # Drop forbidden and irrelevant columns
    df = df.drop(columns=['oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest'], errors='ignore')
    df = df.drop(columns=['nameOrig', 'nameDest'], errors='ignore')

    # One-Hot Encoding
    df = pd.get_dummies(df, columns=['type'], drop_first=True)

    # Handle missing values
    imputer = SimpleImputer(strategy="mean")
    df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

    # Binary target column
    df_imputed['isFraud'] = df_imputed['isFraud'].apply(lambda x: 1 if x > 0 else 0)

    # Features and target
    X = df_imputed.drop(columns=['isFraud'])
    y = df_imputed['isFraud']

    # Scale numerical features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Save preprocessed data
    joblib.dump({"X": X_scaled, "y": y}, processed_file)
    print(f"Preprocessed {'training' if train else 'testing'} data saved!")

    return X_scaled, y

# -------------------- Client Node --------------------
from sklearn.metrics import precision_score, recall_score, f1_score
from tqdm import tqdm  # For progress bar

def client_node():
    print(f"Client {CLIENT_ID}: Starting...")
    global_model_exists = os.path.exists(SERVER_MODEL_PATH)

    if global_model_exists:
        print(f"Client {CLIENT_ID}: Loading Global Model from Server...")
        global_model = joblib.load(SERVER_MODEL_PATH)
    else:
        print(f"Client {CLIENT_ID}: No Global Model Found. Training Fresh Model...")
        global_model = None

    # Load and preprocess training data
    print("Loading training dataset...")
    X_train, y_train = load_and_preprocess_data()
    joblib.dump({"X": X_train, "y": y_train}, "train_data_preprocessed.joblib")
    print("Preprocessed training data saved!")

    # Load and preprocess testing data
    print("Loading testing dataset...")
    TEST_DATASET_PATH = "test_dataset.csv"  # Path to the testing dataset
    X_test, y_test = load_and_preprocess_data()
    joblib.dump({"X": X_test, "y": y_test}, "test_data_preprocessed.joblib")
    print("Preprocessed testing data saved!")

    # Initialize or load the model
    model = RandomForestClassifier(n_estimators=10, warm_start=True, random_state=42)

    # Training Loop with Epochs
    num_epochs = 5
    for epoch in range(1, num_epochs + 1):
        print(f"Client {CLIENT_ID}: Epoch {epoch} - Training...")

        # Incrementally train the model
        model.set_params(n_estimators=epoch * 10)  # Add 10 trees per epoch
        model.fit(X_train, y_train)

        # Evaluate the model on the training dataset
        y_pred_train = model.predict(X_train)
        accuracy = accuracy_score(y_train, y_pred_train)
        precision = precision_score(y_train, y_pred_train)
        recall = recall_score(y_train, y_pred_train)
        f1 = f1_score(y_train, y_pred_train)

        # Print performance metrics
        print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

    # Final Test on the Testing Dataset
    print("Client {CLIENT_ID}: Testing Local Model...")
    y_pred_test = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred_test)
    print(f"Client {CLIENT_ID}: Final Testing Accuracy: {test_accuracy:.4f}")

    # Save the model
    joblib.dump(model, LOCAL_MODEL_PATH)
    print(f"Client {CLIENT_ID}: Local Model Saved Successfully!")

# -------------------- Server Node --------------------
def server_node():
    print("Server: Starting Aggregation...")
    models = []
    for client_id in range(1, 4):  # Assume 3 clients
        model_path = f"client_{client_id}_model.joblib"
        if os.path.exists(model_path):
            model = joblib.load(model_path)
            models.append(model)
            print(f"Server: Model from Client {client_id} Loaded.")

    if not models:
        print("Server: No models found!")
        return

    # Aggregate models
    avg_importances = np.mean([model.feature_importances_ for model in models], axis=0)
    global_model = RandomForestClassifier(n_estimators=100, random_state=42)
    global_model.fit(np.zeros((10, len(avg_importances))), np.zeros(10))  # Dummy data
    global_model.feature_importances_ = avg_importances

    # Save global model
    joblib.dump(global_model, SERVER_MODEL_PATH)
    print("Server: Global Model Aggregated and Saved.")

# -------------------- Execution --------------------
if ROLE == "client":
    client_node()
elif ROLE == "server":
    server_node()

Client 1: Starting...
Client 1: No Global Model Found. Training Fresh Model...
Loading training dataset...
Loading preprocessed training data...
Preprocessed training data saved!
Loading testing dataset...
Loading preprocessed training data...
Preprocessed testing data saved!
Client 1: Epoch 1 - Training...
Accuracy: 0.9998, Precision: 0.9987, Recall: 0.8346, F1 Score: 0.9093
Client 1: Epoch 2 - Training...
Accuracy: 0.9999, Precision: 1.0000, Recall: 0.9130, F1 Score: 0.9545
Client 1: Epoch 3 - Training...
Accuracy: 0.9999, Precision: 1.0000, Recall: 0.9554, F1 Score: 0.9772
Client 1: Epoch 4 - Training...
Accuracy: 1.0000, Precision: 1.0000, Recall: 0.9719, F1 Score: 0.9857
Client 1: Epoch 5 - Training...
Accuracy: 1.0000, Precision: 1.0000, Recall: 0.9835, F1 Score: 0.9917
Client {CLIENT_ID}: Testing Local Model...
Client 1: Final Testing Accuracy: 1.0000
Client 1: Local Model Saved Successfully!
